In [ ]:
from rich import print # always my first import

import random
import datetime
import uuid 

import pandas as pd 
import numpy as np 
import plotly.express as px

from google.cloud import bigquery
from dotenv import load_dotenv

In [ ]:
# all possible dates
initial_date = datetime.date(2022, 1, 1) 
dates = [initial_date + datetime.timedelta(i) for i in range(365)]

# all possible countries with weights
countries = ['GB', 'DE', 'BE', 'FR', 'NL', 'IT', 'ES']
popns = [68, 84, 12, 65, 17, 60, 47]


In [ ]:
# pull together a table of products 
# taken from here: https://support.komoot.com/hc/en-us/articles/360024587532-Komoot-products-and-their-cost
products = pd.DataFrame(
    {
        'product_id': ['p1', 'p2', 'p3', 'p4']
        , 'description': ['Single region', 'Region bundle', 'World pack', 'Premium']
        , 'price': [3.99, 8.99, 29.99, 59.99]
    }
)
product_weights = [50,40,10,5]

In [ ]:
# pull together a table of customers 
custs = [
    {
        'customer_id': uuid.uuid4().__str__() # bigquery doesnt like UUID, wants string
        , 'signup_date': random.choice(dates)
        , 'country': random.choices(countries, popns)[0]
    }
    for i in range(20000)
]
custs = pd.DataFrame(custs)

In [ ]:
# visual check that this distribution is ok.. 
# draw a chart and check youre happy with what that looks like as a distribution
days_after_signup = np.random.default_rng().normal(loc=0, scale = 20, size = 1000)
days_after_signup = [abs(int(i)) for i in days_after_signup]
px.ecdf(days_after_signup)

In [ ]:
# create a purchases table 
purchases = [
    {
        'purchase_id': uuid.uuid4().__str__() # bigquery doesnt like UUID, wants string
        , 'customer_id': random.choice(custs['customer_id'])
        , '_days_after_signup': int(abs(np.random.default_rng().normal(loc=0, scale = 20, size = 1)[0]))
        , 'product_id': random.choices(products['product_id'], product_weights)[0]
        
    }
    for i in range(10000)
]
purchases = pd.DataFrame(purchases)

In [ ]:
# need to backwards engineer the date here. 
purchases = pd.merge(left = purchases, right = custs, on = 'customer_id' )
def create_eventdate(row) : 
    return row['signup_date'] + datetime.timedelta(row['_days_after_signup'])
purchases['eventdate'] = purchases.apply(create_eventdate, axis = 1) 
purchases.drop(['_days_after_signup', 'signup_date', 'country'], axis = 1, inplace = True)

In [ ]:
# following walkthrough here for setting up authentication on g-bq: 
# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
load_dotenv()
client = bigquery.Client()

In [ ]:
# following walkthrough here: https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe
def send_to_bq(df: pd.DataFrame, name: str) -> None : 
    # make sure you have created the "dataset"/schema. i did this via UI, can do programatically tho
    table_id = f'komoot-simon.fakedata.{name}'
    job = client.load_table_from_dataframe(df, table_id, )  # Make an API request.
    return 
send_to_bq(custs, 'customers') 
send_to_bq(products, 'products') 
send_to_bq(purchases, 'purchases') 

In [ ]:
sql = """ 
create or replace view `komoot-simon.reporting.purchases_after_signup` as ( 
  
  with

  purchases as  
    (
      select 
        *
        , row_number() over (partition by customer_id order by eventdate) as event_number 
      from `komoot-simon.fakedata.purchases`
    )

    select 
        a.* 
        , b.* except (customer_id) 
        , c.* except (product_id) 
        , date_diff(b.eventdate, a.signup_date, DAY) as days_since_signup

    from `komoot-simon.fakedata.customers` as a 

    left join purchases as b 
    on a.customer_id = b.customer_id

    left join `komoot-simon.fakedata.products` as c 
    on b.product_id = c.product_id 
  )
;
"""
job = client.query(sql)
job.result() 